In [0]:
# pick the correct table name you created after upload
table_name = "workspace.default.fraud_detection_highacc"  # change if needed

df = spark.table(table_name)
df.printSchema()
display(df.limit(10))
print("Rows:", df.count())


root
 |-- transaction_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- amount: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- location: string (nullable = true)
 |-- is_fraud: long (nullable = true)



transaction_id customer_name amount merchant location is_fraud TXN_1 Ananya Chopra 105795 UnknownMerchant Kolkata 1 TXN_2 Kavya Ghosh 90860 UnknownMerchant Delhi 1 TXN_3 Sneha Reddy 193694 UnknownMerchant Hyderabad 1 TXN_4 Vihaan Singh 166820 UnknownMerchant Delhi 1 TXN_5 Diya Menon 144886 UnknownMerchant Delhi 1 TXN_6 Vihaan Reddy 96265 UnknownMerchant Bangalore 1 TXN_7 Meera Naidu 172386 UnknownMerchant Delhi 1 TXN_8 Nikhil Verma 127194 UnknownMerchant Chennai 1 TXN_9 Sneha Naidu 177498 UnknownMerchant Bangalore 1 TXN_10 Nikhil Nair 134131 UnknownMerchant Mumbai 1

Rows: 500


In [0]:
from pyspark.sql.functions import col

display(df.groupBy("is_fraud").count())
display(df.groupBy("location").count().orderBy(col("count").desc()))
display(df.groupBy("merchant").count().orderBy(col("count").desc()))
display(df.select("amount").summary())


is_fraud count 1 80 0 420

location count Kolkata 99 Mumbai 86 Bangalore 83 Delhi 81 Hyderabad 78 Chennai 73

merchant count LocalShop 135 BankTransfer 84 Amazon 82 Zomato 77 Flipkart 77 UnknownMerchant 45

summary amount count 500 mean 49897.364 stddev 39307.39437184957 min 256 25% 21727 50% 44861 75% 62666 max 196970

In [0]:
from pyspark.sql.functions import log1p

data = (df
        .withColumn("amount_log", log1p(col("amount")))
        .withColumn("label", col("is_fraud").cast("int"))
       )

# drop obvious IDs / high-cardinality text if not used
data = data.drop("transaction_id","customer_name")
display(data.limit(5))


amount merchant location is_fraud amount_log label 105795 UnknownMerchant Kolkata 1 11.569267990508262 1 90860 UnknownMerchant Delhi 1 11.41708614519604 1 193694 UnknownMerchant Hyderabad 1 12.174040035948861 1 166820 UnknownMerchant Delhi 1 12.02467666026271 1 144886 UnknownMerchant Delhi 1 11.88370940723772 1

In [0]:
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)
print("Train:", train_df.count(), "Test:", test_df.count())


Train: 408 Test: 92


In [0]:
from pyspark.ml.feature import FeatureHasher, VectorAssembler

# Hash the categorical columns into a fixed small vector
hasher = FeatureHasher(inputCols=["merchant", "location"], outputCol="cat_hash", numFeatures=512)

train_h = hasher.transform(train_df)
test_h  = hasher.transform(test_df)

# Combine hashed categorical vector with numeric feature(s)
assembler = VectorAssembler(inputCols=["amount_log", "cat_hash"], outputCol="features")

train_prep = assembler.transform(train_h).select("features", "label")
test_prep  = assembler.transform(test_h).select("features", "label")


In [0]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Logistic Regression (compact)
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100, regParam=0.02)
lr_model = lr.fit(train_prep)
pred_lr = lr_model.transform(test_prep)

# Random Forest with controlled size
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100, maxDepth=10)
rf_model = rf.fit(train_prep)
pred_rf = rf_model.transform(test_prep)

# Evaluate
def eval(pred):
    acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
    f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(pred)
    auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(pred)
    return acc, f1, auc

print("LR  (hashed):", eval(pred_lr))
print("RF  (hashed):", eval(pred_rf))


LR  (hashed): (0.9021739130434783, 0.8873166966812451, 0.9634146341463414)
RF  (hashed): (0.8913043478260869, 0.8400799600199899, 0.7274390243902439)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

def evaluate(pred):
    acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
    f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(pred)
    # For AUC we need rawPrediction/probability; set positive class = 1 by default
    auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(pred)
    return acc, f1, auc

acc_lr, f1_lr, auc_lr = evaluate(pred_lr)
acc_rf, f1_rf, auc_rf = evaluate(pred_rf)

print(f"LogReg  -> Accuracy: {acc_lr:.3f} | F1: {f1_lr:.3f} | AUC: {auc_lr:.3f}")
print(f"RandFor -> Accuracy: {acc_rf:.3f} | F1: {f1_rf:.3f} | AUC: {auc_rf:.3f}")


LogReg  -> Accuracy: 0.902 | F1: 0.887 | AUC: 0.963
RandFor -> Accuracy: 0.891 | F1: 0.840 | AUC: 0.727
